# Feed forward (순전파)

Feed forward(순전파)는 Neural net(신경망) 학습의 초기 과정이다.   
순전파를 통해 예측 값(pred_y)을 구하고 예측 값(y_pred)과 정답 값(test_y)의 loss를 구해 weight를 업데이트 하는 과정이 train(학습)이기 때문이다.  
순전파는 미리 정의된 weight matrix와 bias에 입력 데이터를 넣어 dot prodcut (행렬곱)하여 예측 값을 계산한다.  

CF) 학습이 모두 끝난 모델의 weight와 bias를 불러온 후, 새로운 데이터를 넣어 결과 값을 도출하는 과정을 inference라고 한다.

구현하고자 하는 모델 구조는 아래와 같다.
- W1 : 노드 3개
- activate function : sigmoid
- W2 : 노드 2개
- activate function : sigmoid
- W3 : 노드 2개

## Numpy
아래는 numpy를 사용해 구현한 코드이다.

In [1]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def build_model():
    network = {}
    network['W1'] = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['W2'] = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
    network['b2'] = np.array([0.1, 0.2])
    network['W3'] = np.array([[0.1, 0.3], [0.2, 0.4]])
    network['b3'] = np.array([0.1, 0.2])

    return network

def forward(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3

    return a3


model = build_model()
x = np.array([1.0, 0.5])
y = forward(model, x)  # [ 0.31682708  0.69627909]

print(y)

[0.31682708 0.69627909]


## Pytorch
아래는 pytorch를 이용해 구현한 코드이다.  
주목해야할 부분은 torch의 dot product(행렬 곱)은 순서가 바꿔어서 진행이 된다. weight x input data 순서로 진행된다.  
w1은 3x2행렬이고 입력값은 2x1 행렬이다. 둘을 행렬곱 하면 3x1행렬이 된다.  
w2은 2x3행렬이고 w1의 출력값이 3x1행렬이기 때문에 행렬곱하면 2x1 행렬이 된다.  
w3은 2x2행렬이고 w2의 출력값이 2x1행렬이기 때문에 행렬곱하면 2x1 행렬이 된다.  
최종 아웃풋 값은 2x1 행렬을 transpose 값이 출력된다. 

numpy와 다르게 weight 값은 random하게 설정된다.  
물론, 위의 코드처럼 초기 weight 값을 고정할 수 있지만 과정이 번거롭고 역전파 과정을 통해 결국은 weight가 조정되기 때문에 일단은 random하게 설정하는 것만 보겠다.

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FNN(nn.Module):
  def __init__(self):
    super(FNN, self).__init__()

    self.w1 = nn.Linear(2, 3, bias=True)
    self.w2 = nn.Linear(3, 2, bias=True)
    self.w3 = nn.Linear(2, 2, bias=True)
    self.sigmoid = nn.Sigmoid()

    # 내부 weight & bias 보기
    print("weight_1 :", self.w1.weight)
    print("bias_1 :", self.w1.bias)
    print(self.w1.weight.shape, "\n")
    print("weight_2 :", self.w2.weight)
    print("bias_2 :", self.w2.bias)
    print(self.w2.weight.shape, "\n")
    print("weight_3 :", self.w3.weight)
    print("bias_3 :", self.w3.bias)
    print(self.w3.weight.shape, "\n")

  def forward(self, x):
    x = self.w1(x)
    x = self.sigmoid(x)
    x = self.w2(x)
    x = self.sigmoid(x)
    x = self.w3(x)

    return x

random_data = torch.FloatTensor([1, 0])
model = FNN()
print("output =>", model(random_data))

weight_1 : Parameter containing:
tensor([[ 0.5113, -0.3429],
        [-0.2401, -0.3754],
        [ 0.0855,  0.7059]], requires_grad=True)
bias_1 : Parameter containing:
tensor([ 0.3335, -0.5134, -0.5670], requires_grad=True)
torch.Size([3, 2]) 

weight_2 : Parameter containing:
tensor([[-0.2759,  0.1872,  0.2063],
        [-0.4060, -0.2080, -0.4843]], requires_grad=True)
bias_2 : Parameter containing:
tensor([-0.2462, -0.4592], requires_grad=True)
torch.Size([2, 3]) 

weight_3 : Parameter containing:
tensor([[-0.1803,  0.3744],
        [ 0.6408,  0.1686]], requires_grad=True)
bias_3 : Parameter containing:
tensor([-0.2493,  0.5157], requires_grad=True)
torch.Size([2, 2]) 

output => tensor([-0.2249,  0.8338], grad_fn=<AddBackward0>)
